# Here I will simply build a classifier that detects the presence of a red ball or not in the training imagaes.
    - First build classifier
    - Then try to do the sliding window thing.
    - I think the purpose of sliding window is so we know the new location of the ball. E.g. If we find ball in top right then move in that direction.

In [67]:
# imports
import os
import numpy as np
from PIL import Image   # https://www.pluralsight.com/guides/importing-image-data-into-numpy-arrays
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf

In [2]:
# Define dirs
pos_img_dir = 'TrainingImages/positives/'
neg_img_dir = 'TrainingImages/negatives/'

In [3]:
# Follow https://towardsdatascience.com/building-a-convolutional-neural-network-cnn-in-keras-329fbbadc5f5
# Official https://www.tensorflow.org/tutorials/images/cnn


In [4]:
# go through negatives

neg_images = []
neg_labels = np.zeros( len( os.listdir(neg_img_dir) ) )

for filename in os.listdir(neg_img_dir):
    neg_images.append(np.array(Image.open(neg_img_dir + filename))/255 )  # Normal

#plt.imshow(neg_images[0])
np.shape(neg_images)

(2600, 51, 51, 3)

In [5]:
# go through positives

pos_images = []
pos_labels = np.ones( len( os.listdir(pos_img_dir) ) )

for filename in os.listdir(pos_img_dir):
    pos_images.append(np.array(Image.open(pos_img_dir + filename))/255 ) 

#plt.imshow(pos_images[0])
np.shape(pos_images)

(308, 51, 51, 3)

In [6]:
type(pos_images[0])
X = np.concatenate((neg_images, pos_images))

In [68]:
# Now weve got data in array form we can split into test and train data?
# With full colour images

X = np.concatenate((neg_images, pos_images))
y = np.concatenate((neg_labels, pos_labels))

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=300,
                                                    shuffle=True)

# Maybe TODO One-Hot-Encode CNN labels? Does it matter?
#Prove test and train data is split correctly
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
X_train[0].shape

(2326, 51, 51, 3)
(2326,)
(582, 51, 51, 3)
(582,)


(51, 51, 3)

In [69]:
# Multiclass CNN Architecture from 
from tensorflow.keras import datasets, layers, models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(51, 51, 3))) # Or input_shape=(X.shape[1:]) to get inshape
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))
#model.add(layers.Dense(2)) # If activation function is used on last layer then accuracy will only be 0.9 not 0.99  
# Keras Documentation:
# activation: Activation function to use (see activations). 
# If you don't specify anything, no activation is applied (ie. "linear" activation: a(x) = x).

model.summary()

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.fit(X_train, y_train, epochs=2, validation_data=(X_test, y_test))

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 49, 49, 32)        896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 22, 22, 64)        18496     
_________________________________________________________________
flatten_8 (Flatten)          (None, 30976)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                1982528   
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 130       
Total params: 2,002,050
Trainable params: 2,002,050
Non-trainable params: 0
____________________________________________

In [66]:
# Binary Classification CNN Architecture from https://www.tensorflow.org/tutorials/images/cnn
from tensorflow.keras import datasets, layers, models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(51, 51, 3))) # Or input_shape=(X.shape[1:]) to get inshape
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='relu'))
#model.add(layers.Dense(2)) # If activation function is used on last layer then accuracy will only be 0.9 not 0.99  
# Keras Documentation:
# activation: Activation function to use (see activations). 
# If you don't specify anything, no activation is applied (ie. "linear" activation: a(x) = x).

model.summary()

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=2, validation_data=(X_test, y_test))

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 49, 49, 32)        896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 22, 22, 64)        18496     
_________________________________________________________________
flatten_7 (Flatten)          (None, 30976)             0         
_________________________________________________________________
dense_14 (Dense)             (None, 64)                1982528   
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 130       
Total params: 2,002,050
Trainable params: 2,002,050
Non-trainable params: 0
____________________________________________

In [11]:
dave = model.predict(X_test)

In [40]:
dave2 = model.predict(X_test[110:112])
dave2

array([[  7.5748115, -14.36755  ],
       [ -1.8680677,   5.46593  ]], dtype=float32)

In [37]:
y_test[111]

1.0

In [14]:
model.predict_classes(X_test[:20])


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
      dtype=int64)